# Now I'm going to try to translate this book from french to spanish

https://keras.io/examples/nlp/neural_machine_translation_with_keras_hub/

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os

os.chdir('/content/drive/MyDrive/Diplomado/text_processing')
os.listdir()

book = "les_miserables.pdf"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## transforming the original les miserables in french from pdf to txt

In [5]:
!pip install pdfplumber

import pdfplumber

#extracting the text from the pdf and
with pdfplumber.open(book) as pdf:
  meta = pdf.pages
  blen = len(meta)
  print(blen)

  txt = ''
  for i in range(blen):
    first_page = pdf.pages[i]
    text = first_page.extract_text()
    txt += text
    if i == blen:
      print(len(txt))
      pdf.close()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 95.9 MB/s eta 0:00:00
743


In [6]:
with open('les_miserables.txt', 'w') as f:
  f.write(txt)

In [7]:
ls

 _about.txt                les_miserables_translated_from_french.txt   los_miserables.text
 fra-eng.zip               les_miserables.txt                         'Report  - NLP.gdoc'
 fra.txt                   los_miserables.csv                          requirements.txt
 fr_to_esp.tsv             los_miserables.ipynb                        spa-eng.zip
'Keras Translator.ipynb'   los_miserables.parquet                      spa.txt
 les_miserables.pdf        los-miserables.pdf


# Opening the transformed text

In [8]:
with open('les_miserables.txt', 'r') as f:
  txt = f.read()

In [9]:
txt[:10000]

'Victor Hugo\nLLeess MMiisséérraabblleess\nBeQVictor Hugo\nLes Misérables\nPremière partie\nFantine\nLa Bibliothèque électronique du Québec\nCollection À tous les vents\nVolume 648 : version 1.0\n2Du même auteur, à la Bibliothèque :\nLes travailleurs de la mer\nLes derniers jours d’un condamné\nsuivi de Claude Gueux\n3Les Misérables\nÉdition de référence :\nGallimard, Collection Folio Classique.\nLes notes de bas de page appelées par des\nchiffres sont tirées de l’édition de référence ;\ncelles appelées par des lettres, de l’édition\nGallimard, collection de la Pléiade ; celles\nappelées par un astérisque sont de Victor Hugo.\n4Tant qu’il existera, par le fait des lois et des\nmœurs, une damnation sociale créant\nartificiellement, en pleine civilisation, des enfers,\net compliquant d’une fatalité humaine la destinée\nqui est divine ; tant que les trois problèmes du\nsiècle, la dégradation de l’homme par le\nprolétariat, la déchéance de la femme par la faim,\nl’atrophie de l’enfant par 

# motivating me with a pre-trained model -Marian-:

In [10]:
import keras_hub
import pathlib
import random

import keras
from keras import ops

import tensorflow.data as tf_data
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset as bert_vocab,
)

from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer for French to Spanish
model_name = "Helsinki-NLP/opus-mt-fr-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

french_text = "Bonjour tout le monde. Ceci est un test de traduction."
spanish_translation = translate(french_text, tokenizer, model)

print(spanish_translation)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/819k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/332M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/332M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Hola a todos, esta es una prueba de traducción.


trying to translate the fulll text:

In [11]:
import re

# cleaning the text
t_copy = txt

cleanT = re.sub('[!¡@#$-;.,~¿?—\n«»]', '', t_copy.lower().replace('\n', ' '))

# display(cleanT)

# todas las palabras en el set
palabras = cleanT.split()

spanish_txt = translate(cleanT, tokenizer, model)
print(spanish_txt[1000:2000])

iones l'asfixia social será posible en d'embros y complica de una fatalidad humana el destino que es divina el destino que es divina mientras los tres problemas del siglo la degradación del siglo por el proleo la degradación del hombre por el prole por el prole por la pérdida de la pérdida de la mujer por el prole por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre por el hambre po

In [12]:
# saving the text to a file:

with open('les_miserables_translated_from_french.txt', 'w') as f:
  f.write(spanish_txt)

# Now that I'm motivated, I'm tying to built it from scratch

In [13]:
ls

 _about.txt                les_miserables_translated_from_french.txt   los_miserables.text
 fra-eng.zip               les_miserables.txt                         'Report  - NLP.gdoc'
 fra.txt                   los_miserables.csv                          requirements.txt
 fr_to_esp.tsv             los_miserables.ipynb                        spa-eng.zip
'Keras Translator.ipynb'   los_miserables.parquet                      spa.txt
 les_miserables.pdf        los-miserables.pdf


In [14]:
BATCH_SIZE = 64
EPOCHS = 1  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 40
ENG_VOCAB_SIZE = 15000
SPA_VOCAB_SIZE = 15000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

In [15]:
#reading with csv to improve the reading
#cleaning some special characters with regexs

import csv
import re


# in the original tutorial we didn't had any français to español, so I had to pulled directly from https://tatoeba.org/es/downloads
text_file = "fr_to_esp.tsv"

# fr_text_file = "fra.txt"
text_pairs = []

with open("fr_to_esp.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        text_pairs.append((re.sub('[\u202f]', '', row[1]), re.sub('[\u202f]', '', row[3]))) #cleaning


In [16]:
text_pairs[1:10]

[('Je ne supporte pas ce type.', 'No aguanto a este tío.'),
 ('Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.',
  'Por una vez en mi vida hago una buena acción... y no sirve de nada.'),
 ("Ne tenez aucun compte de ce qu'il dit.",
  'No prestéis atención a lo que diga.'),
 ('Essayons quelque chose!', '¡Intentemos algo!'),
 ("Qu'est-ce que tu fais?", '¿Qué estás haciendo?'),
 ("Qu'est-ce que tu fais?", '¿Qué haces?'),
 ("Qu'est-ce que c'est ?", '¿Qué es eso?'),
 ("Qu'est-ce que c'est ?", '¿Qué es esto?'),
 ("Aujourd'hui nous sommes le 18 juin et c'est l'anniversaire de Muiriel!",
  '¡Hoy es 18 de junio y es el cumpleaños de Muiriel!')]

In [17]:
import keras_hub
import pathlib
import random

import keras
from keras import ops

import tensorflow.data as tf_data
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset as bert_vocab,
)

In [18]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

60263 total pairs
42185 training pairs
9039 validation pairs
9039 test pairs


In [19]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_hub.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab

In [20]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_samples = [text_pair[0] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

spa_samples = [text_pair[1] for text_pair in train_pairs]
spa_vocab = train_word_piece(spa_samples, SPA_VOCAB_SIZE, reserved_tokens)

In [21]:
print("English Tokens: ", eng_vocab[100:110])
print("Spanish Tokens: ", spa_vocab[100:110])

English Tokens:  ['é', 'ê', 'ë', 'í', 'î', 'ï', 'ñ', 'ó', 'ô', 'ö']
Spanish Tokens:  ['î', 'ñ', 'ó', 'ö', 'ú', 'ü', 'č', 'đ', 'œ', 'ồ']


In [22]:
eng_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)
spa_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=spa_vocab, lowercase=False
)

In [23]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

print()

spa_input_ex = text_pairs[0][1]
spa_tokens_ex = spa_tokenizer.tokenize(spa_input_ex)
print("Spanish sentence: ", spa_input_ex)
print("Tokens: ", spa_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    spa_tokenizer.detokenize(spa_tokens_ex),
)

English sentence:  Étiez-vous chez vous, hier ?
Tokens:  tf.Tensor([  90 3063   13  151  340  151   12  304   27], shape=(9,), dtype=int32)
Recovered text after detokenizing:  Étiez - vous chez vous , hier ?

Spanish sentence:  ¿Ayer estabais en vuestra casa?
Tokens:  tf.Tensor([  90  461  188  353  125 1110  167   29], shape=(8,), dtype=int32)
Recovered text after detokenizing:  ¿ Ayer estabais en vuestra casa ?


In [24]:
def preprocess_batch(eng, spa):
    batch_size = ops.shape(spa)[0]

    eng = eng_tokenizer(eng)
    spa = spa_tokenizer(spa)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    eng_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `spa` and pad it as well.
    spa_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=spa_tokenizer.token_to_id("[START]"),
        end_value=spa_tokenizer.token_to_id("[END]"),
        pad_value=spa_tokenizer.token_to_id("[PAD]"),
    )
    spa = spa_start_end_packer(spa)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [25]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 40)
inputs["decoder_inputs"].shape: (64, 40)
targets.shape: (64, 40)


In [26]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_hub.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=SPA_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_hub.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(SPA_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

In [27]:
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_and_position_embed… │ (None, None, 256)      │      3,850,240 │ encoder_inputs[0][0]   │
│ (TokenAndPositionEmbeddi… │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      1,315,072 │ token_and_position_em… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_1 (Functional) │ (None, None, 15000)    │      9,283,992 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 14,449,304 (55.12 MB)

 Trainable params: 14,449,304 (55.12 MB)

 Non-trainable params: 0 (0.00 B)

660/660 ━━━━━━━━━━━━━━━━━━━━ 74s 84ms/step - accuracy: 0.7780 - loss: 1.9831 - val_accuracy: 0.8045 - val_loss: 1.1842


In [29]:
test_eng_texts = [pair[0] for pair in test_pairs]

test_eng_texts[:10]

["J'ai entendu dire qu'elle était venue ici.",
 'Le temps a passé très vite.',
 "Qu'y a-t-il dans la caisse?",
 'Est-ce que tu viens avec moi ?',
 'Il y a beaucoup de chevaux dans le voisinage.',
 'Les diamants sont éternels.',
 'Un guépard qui est pressé peut atteindre une vitesse de cent-vingt kilomètres heure.',
 'Les lumières sont allumées jour et nuit.',
 'Vous devriez savourer ce moment.',
 "Ah, la lumière s'est éteinte."]

In [30]:
import numpy as np

def decode_sequences(input_sentences):
    batch_size = 1

    # Tokenize the encoder input.
    encoder_input_tokens = ops.convert_to_tensor(eng_tokenizer(input_sentences))
    if len(encoder_input_tokens[0]) < MAX_SEQUENCE_LENGTH:
        pads = ops.full((1, MAX_SEQUENCE_LENGTH - len(encoder_input_tokens[0])), 0)
        encoder_input_tokens = ops.concatenate(
            [encoder_input_tokens, pads], 1
        )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        # Ignore hidden states for now; only needed for contrastive search.
        hidden_states = None
        return logits, hidden_states, cache

    # Build a prompt of length 40 with a start token and padding tokens.
    length = 40
    start = ops.full((batch_size, 1), spa_tokenizer.token_to_id("[START]"))
    pad = ops.full((batch_size, length - 1), spa_tokenizer.token_to_id("[PAD]"))
    prompt = ops.concatenate((start, pad), axis=-1)

    generated_tokens = keras_hub.samplers.GreedySampler()(
        next,
        prompt,
        stop_token_ids=[spa_tokenizer.token_to_id("[END]")],
        index=1,  # Start sampling after start token.
    )
    generated_sentences = spa_tokenizer.detokenize(generated_tokens)
    return generated_sentences

for i in range(2):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequences([input_sentence])
    translated = np.array(translated[0])
    print(f"** Example {i} **")
    print(input_sentence)
    print(translated)
    print()

** Example 0 **
Tom a fermé les yeux.
[START] No me gusta la ciudad . [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

** Example 1 **
Maintenant ma vie sera meilleure.
[START] Me gusta el mundo . [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]



In [31]:
# trying to decode the full text in french, let's get fun:

In [64]:
# cleaning the text
t_copy = txt

cleanT = re.sub('[!¡@#$-;.,~¿?—\n«»]', '', t_copy.lower().replace('\n', ' '))

In [84]:
translated = decode_sequences([cleanT[100:140]])

In [85]:
translated

['[START] ¿ Qué es un comprecururues de la ciudad ? [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [86]:
cleanT[100:140]

'liothèque électronique du québec collect'